################################################################################################################
# CLASSIFYING 911 CALLS #
################################################################################################################

## DESCRIPTION :: Emergency (911) Calls: Fire, Traffic, EMS for Montgomery County, PA

- You can get a quick introduction to this Dataset with this kernel: Dataset Walk-through @ https://www.kaggle.com/mchirico/d/mchirico/montcoalert/dataset-walk-through-911

## SOURCE :: KAGGLE 
* https://www.kaggle.com/mchirico/montcoalert

## WHY I'M INTERESTED / MY DOMAIN EXPERTISE

I've worked as an EMT and completed paramedic school in 2012.  Two things surprised me from my paramedic internship:
- we spent a lot of time driving from post to post, filling the space left when another unit went on a call
- we responded to a lot of calls that did not need paramedic-level care (needed basic - not advanced - life support based on clinical criteria)
- my preceptor once remarked that she got most calls for dead on arrival in the morning...when someone woke up next to their dead spouse

## WHAT I'M HOPING TO GAIN FROM THIS
- find hotspots & trends for
- - all calls
- - specific types of calls
- - time
- - level of response requested

> # BASIC PACKAGES IMPORTS

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

> # DATA IMPORT

In [5]:
data = pd.read_csv('http://localhost:8889/files/project/911.csv/911.csv')
data.shape

(120518, 9)

### overview

In [7]:
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120518 entries, 0 to 120517
Data columns (total 9 columns):
lat          120518 non-null float64
lng          120518 non-null float64
desc         120518 non-null object
zip          104887 non-null float64
title        120518 non-null object
timeStamp    120518 non-null object
twp          120467 non-null object
addr         119910 non-null object
e            120518 non-null int64
dtypes: float64(3), int64(1), object(5)
memory usage: 8.3+ MB


lat              0
lng              0
desc             0
zip          15631
title            0
timeStamp        0
twp             51
addr           608
e                0
dtype: int64

### first look

In [9]:
data.head(20)

,lat,lng,desc,zip,title,timeStamp,twp,addr,e
0,40.297876,-75.581294,REINDEER CT & DEAD END; NEW HANOVER; Station ...,19525.0,EMS: BACK PAINS/INJURY,2015-12-10 17:40:00,NEW HANOVER,REINDEER CT & DEAD END,1
1,40.258061,-75.264680,BRIAR PATH & WHITEMARSH LN; HATFIELD TOWNSHIP...,19446.0,EMS: DIABETIC EMERGENCY,2015-12-10 17:40:00,HATFIELD TOWNSHIP,BRIAR PATH & WHITEMARSH LN,1
2,40.121182,-75.351975,HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...,19401.0,Fire: GAS-ODOR/LEAK,2015-12-10 17:40:00,NORRISTOWN,HAWS AVE,1
3,40.116153,-75.343513,AIRY ST & SWEDE ST; NORRISTOWN; Station 308A;...,19401.0,EMS: CARDIAC EMERGENCY,2015-12-10 17:40:01,NORRISTOWN,AIRY ST & SWEDE ST,1
4,40.251492,-75.603350,CHERRYWOOD CT & DEAD END; LOWER POTTSGROVE; S...,NaN,EMS: DIZZINESS,2015-12-10 17:40:01,LOWER POTTSGROVE,CHERRYWOOD CT & DEAD END,1
5,40.253473,-75.283245,CANNON AVE & W 9TH ST; LANSDALE; Station 345;...,19446.0,EMS: HEAD INJURY,2015-12-10 17:40:01,LANSDALE,CANNON AVE & W 9TH ST,1
6,40.182111,-75.127795,LAUREL AVE & OAKDALE AVE; HORSHAM; Station 35...,19044.0,EMS: NAUSEA/VOMITING,2015-12-10 17:40:01,HORSHAM,LAUREL AVE & OAKDALE AVE,1
7,40.217286,-75.405182,COLLEGEVILLE RD & LYWISKI RD; SKIPPACK; Stati...,19426.0,EMS: RESPIRATORY EMERGENCY,2015-12-10 17:40:01,SKIPPACK,COLLEGEVILLE RD & LYWISKI RD,1
8,40.289027,-75.399590,MAIN ST & OLD SUMNEYTOWN PIKE; LOWER SALFORD;...,19438.0,EMS: SYNCOPAL EPISODE,2015-12-10 17:40:01,LOWER SALFORD,MAIN ST & OLD SUMNEYTOWN PIKE,1
9,40.102398,-75.291458,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD; PLYM...,19462.0,Traffic: VEHICLE ACCIDENT -,2015-12-10 17:40:01,PLYMOUTH,BLUEROUTE & RAMP I476 NB TO CHEMICAL RD,1


In [10]:
data.desc

0         REINDEER CT & DEAD END;  NEW HANOVER; Station ...
1         BRIAR PATH & WHITEMARSH LN;  HATFIELD TOWNSHIP...
2         HAWS AVE; NORRISTOWN; 2015-12-10 @ 14:39:21-St...
3         AIRY ST & SWEDE ST;  NORRISTOWN; Station 308A;...
4         CHERRYWOOD CT & DEAD END;  LOWER POTTSGROVE; S...
5         CANNON AVE & W 9TH ST;  LANSDALE; Station 345;...
6         LAUREL AVE & OAKDALE AVE;  HORSHAM; Station 35...
7         COLLEGEVILLE RD & LYWISKI RD;  SKIPPACK; Stati...
8         MAIN ST & OLD SUMNEYTOWN PIKE;  LOWER SALFORD;...
9         BLUEROUTE  & RAMP I476 NB TO CHEMICAL RD; PLYM...
10        RT202 PKWY & KNAPP RD; MONTGOMERY; 2015-12-10 ...
11        BROOK RD & COLWELL LN; PLYMOUTH; 2015-12-10 @ ...
12        BYBERRY AVE & S WARMINSTER RD; UPPER MORELAND;...
13        OLD YORK RD & VALLEY RD; CHELTENHAM; 2015-12-1...
14        SCHUYLKILL EXPY & CROTON RD UNDERPASS; UPPER M...
15        STUMP RD & WITCHWOOD DR; MONTGOMERY; 2015-12-1...
16        SUSQUEHANNA AVE & W MAIN ST;  

# NOTES

- desc includes cross streets; town; maybe zip;  || all separated by semi-colon
- - need to split column to drill down
- Might be able to fix the null zips with the lat/long
- Need to change zips to 5-digit strings
- Title :: split at the ':' to categorize fire / ems
- split addr up to see about many incidents on some streets (easier ems commute)
- fix time stamps to each level
- drop last column 'e'
- maybe can use lat/lng to determine radiuses & time between calls in that area


## Ideas
- some way to map these once categorized?
- any way to animate it over time?
- are there any trends for the types of calls? ems / fire / types of chief complaint?


### CLASSIFY based on 
- ems / fire, etc
- town name ('twp')
- further split the calls into chief complaints
- time of day, day of week, month, etc


### GOAL :: Recommend geographical & time staffing for each type of emergency (fire, ems, trauma, ALS) 

### basic cleaning

In [11]:
# data.columns = [each.replace('-','_').lower() for each in data.columns]
# data = data.rename(columns = {'a1cresult' : 'a1c', 'diabetesmed' : 'diabetes_med' })